---
# Objetivo: Treinar modelo BERT no conjunto de dados filtrado pela categoria Animal. Variavel target o tema dessa categoria. Ex. Gado, Suínos, Ovinos
---

**1- Descrição do problema ou tarefa:**

Prever o tema da categoria Animal a partir das perguntas.

**2- Descrição da solução de IA:**
Treinamento supervisionado de modelo de classificação os temas da categoria Animal (6 classes) com os dados de 4029 perguntas.

**3- Fonte de dados:**

Livro de perguntas e respostas da Embrapa
https://mais500p500r.sct.embrapa.br/view/index.php4

**4-Variáveis independentes:** 
perguntas

**5- Variável dependente:** 
Temas da cagetoria animal: Gado, Suínos, Ovinos...

-----------------------------------------------------------------
Autor do caderno: Wellington Rangel
Data: 23/11/2021

In [ ]:
# install ktrain on Google Colab
!pip3 install ktrain

     |████████████████████████████████| 25.3 MB 1.6 MB/s 
     |████████████████████████████████| 6.8 MB 34.3 MB/s 
     |████████████████████████████████| 981 kB 34.8 MB/s 
     |████████████████████████████████| 263 kB 40.8 MB/s 
     |████████████████████████████████| 2.8 MB 29.8 MB/s 
     |████████████████████████████████| 1.2 MB 37.2 MB/s 
     |████████████████████████████████| 468 kB 40.0 MB/s 
     |████████████████████████████████| 596 kB 33.2 MB/s 
     |████████████████████████████████| 895 kB 40.4 MB/s 
     |████████████████████████████████| 61 kB 438 kB/s 
     |████████████████████████████████| 3.3 MB 28.9 MB/s 
  Created wheel for ktrain: filename=ktrain-0.28.3-py3-none-any.whl size=25292659 sha256=80bfd9ece22147931edc0967949b5500f9d3ffac4dd7e6329d59d60da0ecccbc
  Stored in directory: /root/.cache/pip/wheels/6a/7e/c3/f46cdfc2b81c54424923b1405d7e670c35cacc11ada9a47b1c
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=204400c0be2a7fc1

In [ ]:
# import ktrain and the ktrain.text modules
import ktrain
from ktrain import text

In [ ]:
ktrain.__version__

'0.28.3'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/projeto

/content/drive/MyDrive/Colab Notebooks/projeto


In [ ]:
import pandas as pd
df = pd.read_csv('dados_agrupados.csv',  sep=',', low_memory=False,encoding='latin-1' )


In [ ]:
df

,Numero,Livro,Capitulo,Pergunta,Resposta,Target,Target_final,Target_final1,Target_final2,Target_final3
0,1,Pera,Generalidades,Qual é o centro de origem da pereira?,São citados três centros de origem da pereira:...,Pera,Frutas,NaN,Frutas,NaN
1,2,Pera,Generalidades,Qual é o centro de origem mais importante?,O centro do Oriente Médio é considerado de imp...,Pera,Frutas,NaN,Frutas,NaN
2,3,Pera,Generalidades,Como ocorreu a disseminação da pereira pelo mu...,"Com base em estudos bioquímicos, verificou-se ...",Pera,Frutas,NaN,Frutas,NaN
3,4,Pera,Generalidades,Quais são as espécies de pereira mais cultivad...,"Na Europa, na América do Norte, na América do ...",Pera,Frutas,NaN,Frutas,NaN
4,5,Pera,Generalidades,Quando a pereira foi introduzida no Brasil?,Não há relatos na literatura sobre a introduçã...,Pera,Frutas,NaN,Frutas,NaN
...,...,...,...,...,...,...,...,...,...,...
16591,497,Uva,NaN,Qual é a produção nacional e a mundial de vinhos?,O Brasil produz cerca de 300 milhões de litros...,Uva,Frutas,NaN,Frutas,NaN
16592,498,Uva,NaN,Quais são os maiores produtores e os maiores e...,"Os maiores produtores são a França, a Espanha ...",Uva,Frutas,NaN,Frutas,NaN
16593,499,Uva,NaN,Que quantidade de vinho o Brasil importa anual...,"Mais de 55 milhões de litros de vinho fino, o ...",Uva,Frutas,NaN,Frutas,NaN
16594,500,Uva,NaN,Qual é o consumo per capita dos países que mai...,"França e Luxemburgo, 55 litros; Portugal, 48 l...",Uva,Frutas,NaN,Frutas,NaN


In [ ]:
df = df.loc[df['Target_final'] == 'Animal']

In [ ]:

df = df[['Pergunta', 'Target']]


In [ ]:
df.head()

,Pergunta,Target
4518,O que são búfalos domésticos e qual sua origem?,Búfalos
4519,"Bubalus arnee por Keer, em 1792. Como se orig...",Búfalos
4520,Onde e quando foram domesticados?,Búfalos
4521,Qual a versão mais provável da introdução dos ...,Búfalos
4522,Como são classificados zoologicamente os búfalos?,Búfalos


In [ ]:
df.tail()

,Pergunta,Target
16090,o que é um criador de ciclo completo?,Suínos
16091,O que é um criador de leitões?,Suínos
16092,O que é um terminador?,Suínos
16093,O que é um criador de reprodutores?,Suínos
16094,Quais as alternativas para reduzir o custo de ...,Suínos


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')


In [ ]:
sub = df[[ 'Pergunta', 'Target']]

In [ ]:
sub['texto_sem_stopwords'] = sub['Pergunta'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [ ]:
sub = sub.dropna()

In [ ]:
sub

,Pergunta,Target,texto_sem_stopwords
4518,O que são búfalos domésticos e qual sua origem?,Búfalos,O búfalos domésticos origem?
4519,"Bubalus arnee por Keer, em 1792. Como se orig...",Búfalos,"Bubalus arnee Keer, 1792. Como originaram raça..."
4520,Onde e quando foram domesticados?,Búfalos,Onde domesticados?
4521,Qual a versão mais provável da introdução dos ...,Búfalos,Qual versão provável introdução búfalos Brasil?
4522,Como são classificados zoologicamente os búfalos?,Búfalos,Como classificados zoologicamente búfalos?
...,...,...,...
16090,o que é um criador de ciclo completo?,Suínos,criador ciclo completo?
16091,O que é um criador de leitões?,Suínos,O criador leitões?
16092,O que é um terminador?,Suínos,O terminador?
16093,O que é um criador de reprodutores?,Suínos,O criador reprodutores?


In [ ]:
# Separa os dados de treino e teste (proporção 80/20, stratify para classes desbalanceadas)
from sklearn.model_selection import train_test_split
m_X_train, m_X_test, m_y_train, m_y_test = train_test_split(df.Pergunta, df.Target, test_size=0.2, 
                                                    stratify=df.Target,random_state=42)




In [ ]:
type(m_X_train)

pandas.core.series.Series

In [ ]:
m_y_train

7663                                 Gado
8597                                 Gado
16046                              Suínos
7955                                 Gado
8718                                 Gado
                       ...               
13661                              Ovinos
14635    Pesca e Piscicultura no Pantanal
13906                              Ovinos
12043          Caprinos e Ovinos de Corte
8968                                 Gado
Name: Target, Length: 3223, dtype: object

In [ ]:
x_train = m_X_train.tolist()
y_train = m_y_train.tolist()
x_test = m_X_test.tolist()
y_test = m_y_test.tolist()

In [ ]:
type(x_train)

list

In [ ]:

type(x_train)


list

In [ ]:
classes = df.Target.unique().tolist()

In [ ]:
type(classes)

list

In [ ]:
classes

['Búfalos',
 'Gado',
 'Caprinos e Ovinos de Corte',
 'Ovinos',
 'Pesca e Piscicultura no Pantanal',
 'Suínos']

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=classes,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

downloading pretrained BERT model (multi_cased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: pt


Is Multi-Label? False
preprocessing test...
language: pt


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:621: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


task: text classification


## Treinando o modelo



In [ ]:
# you can disregard the deprecation warnings arising from using Keras 2.2.4 with TensorFlow 1.14.
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=8)

Is Multi-Label? False
maxlen is 350
done.


In [ ]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
403/403 [==============================] - 555s 1s/step - loss: 1.3193 - accuracy: 0.5197
Epoch 2/4
403/403 [==============================] - 529s 1s/step - loss: 0.6873 - accuracy: 0.7605
Epoch 3/4
403/403 [==============================] - 529s 1s/step - loss: 0.4294 - accuracy: 0.8514
Epoch 4/4
403/403 [==============================] - 530s 1s/step - loss: 0.1867 - accuracy: 0.9376


In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=classes)

                                  precision    recall  f1-score   support

                         Búfalos       0.98      0.84      0.90       100
                            Gado       0.81      0.72      0.76       100
      Caprinos e Ovinos de Corte       0.85      0.91      0.88       304
                          Ovinos       0.83      0.90      0.87       100
Pesca e Piscicultura no Pantanal       0.94      0.84      0.89       101
                          Suínos       0.78      0.81      0.80       101

                        accuracy                           0.86       806
                       macro avg       0.87      0.84      0.85       806
                    weighted avg       0.86      0.86      0.86       806



array([[ 84,   2,   9,   2,   1,   2],
       [  0,  72,  18,   5,   0,   5],
       [  2,   6, 278,   5,   4,   9],
       [  0,   4,   4,  90,   0,   2],
       [  0,   1,  10,   0,  85,   5],
       [  0,   4,   9,   6,   0,  82]])

## Usando o modelo treinado


In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.get_classes()

['Búfalos',
 'Caprinos e Ovinos de Corte',
 'Gado',
 'Ovinos',
 'Pesca e Piscicultura no Pantanal',
 'Suínos']

In [ ]:
df[0:1]

,Pergunta,Target
4518,O que são búfalos domésticos e qual sua origem?,Búfalos


In [ ]:
predictor.predict(df[0:1]['Pergunta'].to_string())

'Búfalos'

In [ ]:
df[0:1]['Pergunta']

4518    O que são búfalos domésticos e qual sua origem?
Name: Pergunta, dtype: object

In [ ]:
# we can visually verify that our prediction of 'sci.med' for this document is correct
print(df[0:1]['Target'])

4518    Búfalos
Name: Target, dtype: object


In [ ]:
predictor.save('/content/drive/MyDrive/Colab Notebooks/projeto/modelos/my_predictor_animais')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
